# 目的
過去1年の平均と現在の価格の差と、  
その後の価格の変化を調査する。    
価格は平均に戻ろうとするのではないかと予想する

# 方法
2013年から2020年まで、1ケ月毎にランダムに日付を抽出(日付の偏りを避けるため)  
その日をベースに過去1年の終値の平均を算出、終値が標準偏差いくつ分下げているかで銘柄をカテゴライズ

カテゴリー毎にその後半年での値動きを確認
  
※ 値動きは、30日間の移動平均を日ごとにプロットし、  
その近似直線の傾きにて、値上がり・値下がりを判断する

In [1]:
%matplotlib inline 

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [2]:
import sys
sys.path.append('../src')

### データの読込

In [3]:
from data_control import data as dt
from analyze import graph as gh
from database.models import DailyChart

In [4]:
bgn_datetime = datetime(2012, 7, 1)
end_datetime = datetime(2021, 6, 30)

In [5]:
all_data = DailyChart.date_between(bgn_datetime - relativedelta(months=2), end_datetime)

2021-11-12 03:29:06,658 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'sql_mode'
2021-11-12 03:29:06,659 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-12 03:29:06,665 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2021-11-12 03:29:06,666 INFO sqlalchemy.engine.Engine [generated in 0.00114s] {}
2021-11-12 03:29:06,670 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2021-11-12 03:29:06,670 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-12 03:29:06,673 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-12 03:29:06,681 INFO sqlalchemy.engine.Engine SELECT daily_chart.chart_date AS daily_chart_chart_date, daily_chart.description_code AS daily_chart_description_code, daily_chart.open AS daily_chart_open, daily_chart.high AS daily_chart_high, daily_chart.low AS daily_chart_low, daily_chart.close AS daily_chart_close, daily_chart.turnover AS daily_chart_turnover, daily_chart.vwap AS daily_chart_vwap, daily_chart.execution_count AS daily_chart_execut

In [6]:
data = dt.separate_by_code(all_data)

In [7]:
data = dt.calc_moving_average(data, days=30, column='close')

In [8]:
# 銘柄数
len(data.keys())

4116

### 調査に必要な関数を定義

In [9]:
def get_one_year_ago(date: datetime) -> datetime:
    """
    半年前の日付を取得する

    Params
    -------
    date: datetime
        基準日

    Returns
    -------
    0: datetime
        半年前の日付
    """
    delta = relativedelta(years=1)
    a_day = relativedelta(days=1)
    return date - delta + a_day

In [10]:
def get_half_year_after(date: datetime) -> datetime:
    """
    半年後の日付を取得する

    Params
    -------
    date: datetime
        基準日

    Returns
    -------
    0: datetime
        半年後の日付
    """
    delta = relativedelta(months=6)
    return date + delta

In [11]:
def calc_mean_and_gap(date: datetime, data: dict) -> dict:
    """
    与えられた基準日をベースに、
    過去半年間の終値の平均、標準偏差と
    その平均と基準日の終値との差を標準偏差で除した値を算出する

    Params
    -------
    date: datetime
        基準日
    data: dict
        銘柄コードをkeyにした価格データ

    Returns
    --------
    0: dict
        算出した値を列として保持したデータ
        (mean, std, code, now, gap, gap_rate)
    """
    df = pd.DataFrame()
    half_year_ago = get_one_year_ago(date)
    for code, d in data.items():
        d = d[(d['chart_date'] >= half_year_ago) & (d['chart_date'] <= date)]
        if len(d) > 0:
            describe = pd.DataFrame([d['close'].describe()[['mean', 'std']]])
            describe['code'] = code
            describe['now'] = float(d.sort_values('chart_date').tail(1)['close'])
            df = pd.concat([df, describe])
    df = df.reset_index().drop('index', axis=1)
    df['gap'] = df['now'] - df['mean']
    df['gap_rate'] = df['gap'] / df['std']

    return df

In [12]:
def get_random_date(from_dt: datetime, end_dt: datetime) -> datetime:
    """
    与えられた日付間でランダムに1件日付を抽出する

    Params
    -------
    from_dt: datetime
        抽出開始日
    end_dt: datetime
        抽出終了日
    Returns
    -------
    0: 抽出した日付
        
    """
    days = (end_dt - from_dt).days + 1
    delta = timedelta(days=np.random.randint(0, days))
    return from_dt + delta

In [13]:
def check(code_list: list, date: datetime, show_graph=False) -> pd.DataFrame:
    """
    基準日から先の日付で値上がり、値下がりする銘柄を調べる

    Params
    -------
    code_list: list
        対象とする銘柄コード
    date: datetime
        基準日
    show_graph: bool 
        価格変動のグラフを出力するかの真偽値
    Returns
    -------
    0: 調査結果のDataFrame(
        grad: 価格近似直線の傾き
        max_profit: 最高益
        max_days: 最高益地点までの日数
        min_profit: 最高損
        min_days: 最高損までの日数
        )
    
    """
    grad_list = list()
    max_list = list()
    max_days_list = list()
    min_list = list()
    min_days_list = list()

    for code in code_list:
        # データの絞り込み
        d = data[code]
        d = d[(date < d['chart_date']) & (d['chart_date'] <= get_half_year_after(date))]
        d = d.reset_index()
        d = d.dropna()
        if len(d) == 0:
            continue
        # 回帰直線の追加
        x = np.linspace(1, len(d), len(d))
        a, b = np.polyfit(x, d['m_avg_30_close'], 1)
        regression_line = a * x + b
        d = d.assign(regression=regression_line)
        # 損益の取得
        buy = float(d.head(1)['close'])
        std = float(d.head(1)['m_std_30_close'])
        if std == 0:
            std = 0.0001
        d['profit_or_loss'] = d['m_avg_30_close'] - buy
        max_idx = d['profit_or_loss'].idxmax()
        min_idx = d['profit_or_loss'].idxmin()
        max_by_std = float(d[d.index==max_idx]['profit_or_loss']) / std
        min_by_std = float(d[d.index==min_idx]['profit_or_loss']) / std
        # 集計
        grad_list.append(a)
        max_list.append(max_by_std)
        max_days_list.append(max_idx)
        min_list.append(min_by_std)
        min_days_list.append(min_idx)
        # グラフの描画
        if show_graph:
            print(code)
            gh.show_graph_together(d, ['m_avg_30_close', 'regression'], ['b', 'r'])
    
    check_df = pd.DataFrame()
    check_df = check_df.assign(grad=grad_list)
    check_df = check_df.assign(max_profit=max_list)
    check_df = check_df.assign(max_days=max_days_list)
    check_df = check_df.assign(min_profit=min_list)
    check_df = check_df.assign(min_days=min_days_list)
    return check_df

### 調査の実行

In [14]:
# 1か月ごとに日付を取得
term = relativedelta(days=30)
a_day = relativedelta(days=1)
base_datetime = bgn_datetime
check_datetime_list = []
while base_datetime <= end_datetime:
    to_datetime = min(base_datetime + term, end_datetime)
    check_datetime_list.append(get_random_date(base_datetime, to_datetime))
    base_datetime = base_datetime + term

In [15]:
gap_rank = [(-99.0, 99.0), (-99.0, -2.0), (-2.0, -1.5), (-1.5, -1.0), (-1.0, -0.5), (-0.5, 0), (0, 0.5), (0.5, 1.0), (1.0, 1.5), (1.5, 2.0), (2.0, 99.0)]
for check_datetime in check_datetime_list:
    gap_df = calc_mean_and_gap(check_datetime, data)
    for rank in gap_rank:
        code_list = gap_df[(rank[0] < gap_df['gap_rate']) & (gap_df['gap_rate'] <= rank[1])].code
        check_df = check(code_list, check_datetime)
        plus = len(check_df[check_df['grad'] > 0])
        minus = len(check_df[check_df['grad'] < 0])
        print('{}, ({}, {}): plus => {}, minus => {}'.format(check_datetime, rank[0], rank[1], plus / len(check_df), minus / len(check_df)))

<ipython-input-15-f02c003211c5>:6: RankWarning: Polyfit may be poorly conditioned
  check_df = check(code_list, check_datetime)
<ipython-input-15-f02c003211c5>:6: RankWarning: Polyfit may be poorly conditioned
  check_df = check(code_list, check_datetime)


2012-07-19 00:00:00, (-99.0, 99.0): plus => 0.5818965517241379, minus => 0.41810344827586204
2012-07-19 00:00:00, (-99.0, -2.0): plus => 0.41379310344827586, minus => 0.5862068965517241
2012-07-19 00:00:00, (-2.0, -1.5): plus => 0.5, minus => 0.5
2012-07-19 00:00:00, (-1.5, -1.0): plus => 0.5328185328185329, minus => 0.4671814671814672
2012-07-19 00:00:00, (-1.0, -0.5): plus => 0.552863436123348, minus => 0.44713656387665196
2012-07-19 00:00:00, (-0.5, 0): plus => 0.5544354838709677, minus => 0.44556451612903225


<ipython-input-15-f02c003211c5>:6: RankWarning: Polyfit may be poorly conditioned
  check_df = check(code_list, check_datetime)


2012-07-19 00:00:00, (0, 0.5): plus => 0.5726141078838174, minus => 0.42738589211618255
2012-07-19 00:00:00, (0.5, 1.0): plus => 0.625560538116592, minus => 0.3744394618834081
2012-07-19 00:00:00, (1.0, 1.5): plus => 0.6386138613861386, minus => 0.3613861386138614


<ipython-input-15-f02c003211c5>:6: RankWarning: Polyfit may be poorly conditioned
  check_df = check(code_list, check_datetime)


...

2021-06-26 00:00:00, (0, 0.5): plus => 0.5656934306569343, minus => 0.4343065693430657


<ipython-input-15-f02c003211c5>:6: RankWarning: Polyfit may be poorly conditioned
  check_df = check(code_list, check_datetime)


2021-06-26 00:00:00, (0.5, 1.0): plus => 0.602291325695581, minus => 0.397708674304419
2021-06-26 00:00:00, (1.0, 1.5): plus => 0.7368421052631579, minus => 0.2631578947368421


<ipython-input-15-f02c003211c5>:6: RankWarning: Polyfit may be poorly conditioned
  check_df = check(code_list, check_datetime)


2021-06-26 00:00:00, (1.5, 2.0): plus => 0.9134396355353075, minus => 0.08656036446469248
2021-06-26 00:00:00, (2.0, 99.0): plus => 0.967741935483871, minus => 0.03225806451612903


# 結論
全体的には平均より値が上がっている銘柄がその後も値上がりしやすい傾向  
ただし、日によってかなりムラがあり、  
どの銘柄を買うかより、むしろどの日に買うかの方が重要  
  
特にみられる傾向としては、全体的に値下がりしている日には、  
平均より上の銘柄がより、上がりやすい  
（ただそれより、どの日に買うかという方が重要）  
  
また値上がりのチェック期間を半年としているが、  
これが妥当かいくつかの期間で試すべきと思われる